<a href="https://colab.research.google.com/github/Mercymerine/decision_tree/blob/main/Covid_Decision_tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Getting the necessary  packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Getting the dataset

In [ ]:
#Getting the dataset from kaggle
!kaggle datasets download -d kallolnath1/covid-tested-patient-dataset

Dataset URL: https://www.kaggle.com/datasets/kallolnath1/covid-tested-patient-dataset
License(s): unknown
covid-tested-patient-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
import zipfile
zipfile_path = '/content/covid-tested-patient-dataset.zip'
csv_file = zipfile_path.replace('zip', '')
with zipfile.ZipFile(zipfile_path, 'r') as file:
  file.extractall()
  print('Done')

Done


In [ ]:
covid = pd.read_csv('/content/corona_tested_006.csv', low_memory=False)
covid.head()

,Ind_ID,Test_date,Cough_symptoms,Fever,Sore_throat,Shortness_of_breath,Headache,Corona,Age_60_above,Sex,Known_contact
0,1,11-03-2020,True,False,True,False,False,negative,NaN,NaN,Abroad
1,2,11-03-2020,False,True,False,False,False,positive,NaN,NaN,Abroad
2,3,11-03-2020,False,True,False,False,False,positive,NaN,NaN,Abroad
3,4,11-03-2020,True,False,False,False,False,negative,NaN,NaN,Abroad
4,5,11-03-2020,True,False,False,False,False,negative,NaN,NaN,Contact with confirmed


In [ ]:
covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278848 entries, 0 to 278847
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   Ind_ID               278848 non-null  int64 
 1   Test_date            278848 non-null  object
 2   Cough_symptoms       278596 non-null  object
 3   Fever                278596 non-null  object
 4   Sore_throat          278847 non-null  object
 5   Shortness_of_breath  278847 non-null  object
 6   Headache             278847 non-null  object
 7   Corona               278848 non-null  object
 8   Age_60_above         151528 non-null  object
 9   Sex                  259285 non-null  object
 10  Known_contact        278848 non-null  object
dtypes: int64(1), object(10)
memory usage: 23.4+ MB


## Data Cleaning

In [ ]:
covid.nunique()

,0
Ind_ID,278848
Test_date,51
Cough_symptoms,2
Fever,2
Sore_throat,2
Shortness_of_breath,2
Headache,2
Corona,3
Age_60_above,2
Sex,2


In [ ]:
#percentage of missing values
covid.isnull().sum()/len(covid)*100

,0
Ind_ID,0.000000
Test_date,0.000000
Cough_symptoms,0.090372
Fever,0.090372
Sore_throat,0.000359
Shortness_of_breath,0.000359
Headache,0.000359
Corona,0.000000
Age_60_above,45.659284
Sex,7.015650


In [ ]:
#Dropping the age_60 and above column
columns = ['Age_60_above', 'Ind_ID', 'Test_date']

covid.drop(columns, axis=1, inplace=True)

In [ ]:
#Dropping all rows with missing values
covid.dropna(inplace=True)

In [ ]:
covid.isnull().sum()

,0
Cough_symptoms,0
Fever,0
Sore_throat,0
Shortness_of_breath,0
Headache,0
Corona,0
Sex,0
Known_contact,0


## Encoding

In [ ]:
covid['Corona'].unique()

array(['negative', 'positive', 'other'], dtype=object)

In [ ]:
#Applying label encoding to the target column

covid_encoding ={
    'positive':1,
    'negative':0,
    'other':-1
    }

covid['Corona'] = covid['Corona'].map(covid_encoding)
covid

,Cough_symptoms,Fever,Sore_throat,Shortness_of_breath,Headache,Corona,Sex,Known_contact
13727,True,False,False,False,False,0,male,Abroad
13728,True,False,False,False,False,0,female,Abroad
13730,True,True,False,False,False,1,female,Other
13731,False,False,False,False,False,-1,female,Other
13732,False,False,False,False,False,-1,female,Other
...,...,...,...,...,...,...,...,...
278843,False,False,False,False,False,1,male,Other
278844,False,False,False,False,False,0,female,Other
278845,False,False,False,False,False,0,male,Other
278846,False,False,False,False,False,0,male,Other


In [ ]:
#Applying encoding using get dummies in pandas
covid = pd.get_dummies(covid, drop_first=True).astype(int)
covid

,Corona,Cough_symptoms_True,Fever_True,Sore_throat_True,Shortness_of_breath_True,Headache_True,Sex_male,Known_contact_Contact with confirmed,Known_contact_Other
13727,0,1,0,0,0,0,1,0,0
13728,0,1,0,0,0,0,0,0,0
13730,1,1,1,0,0,0,0,0,1
13731,-1,0,0,0,0,0,0,0,1
13732,-1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...
278843,1,0,0,0,0,0,1,0,1
278844,0,0,0,0,0,0,0,0,1
278845,0,0,0,0,0,0,1,0,1
278846,0,0,0,0,0,0,1,0,1


## Splitting the datset into training and test data

In [ ]:
from sklearn.model_selection import train_test_split

X = covid.drop('Corona', axis=1)
y = covid['Corona']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Modelling

### Decision tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
dt = DecisionTreeClassifier(criterion='gini')


In [ ]:
# Fit the model on the training data
dt.fit(X_train, y_train)

# Predict on the test set
y_pred = dt.predict(X_test)

# Evaluate the model on the test set
test_accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", test_accuracy)

# Additional performance metrics
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Test Accuracy: 0.9575556370514775
Classification Report:
               precision    recall  f1-score   support

          -1       0.00      0.00      0.00       631
           0       0.97      0.99      0.98     48522
           1       0.76      0.58      0.66      2656

    accuracy                           0.96     51809
   macro avg       0.58      0.52      0.55     51809
weighted avg       0.94      0.96      0.95     51809

Confusion Matrix:
 [[    0   604    27]
 [    0 48070   452]
 [    0  1116  1540]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rfc = RandomForestClassifier(criterion='gini')


In [ ]:
# Fit the model on the training data
rfc.fit(X_train, y_train)

# Predict on the test set
y_pred = rfc.predict(X_test)

# Evaluate the model on the test set
test_accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", test_accuracy)

# Additional performance metrics
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Test Accuracy: 0.9575556370514775
Classification Report:
               precision    recall  f1-score   support

          -1       0.00      0.00      0.00       631
           0       0.97      0.99      0.98     48522
           1       0.76      0.58      0.66      2656

    accuracy                           0.96     51809
   macro avg       0.58      0.52      0.55     51809
weighted avg       0.94      0.96      0.95     51809

Confusion Matrix:
 [[    0   604    27]
 [    0 48069   453]
 [    0  1115  1541]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
